# Mesh generation for tutorial 06, case 3b

This file generates the mesh which is used in the following examples:
* 3b_advection_diffusion_reaction_neumann_control

The test case is from section 5.3 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import gmsh
from mpi4py import MPI
from dolfinx.io import XDMFFile
from multiphenicsx.mesh.utils import gmsh_to_fenicsx

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters and related quantities

In [ ]:
lcar = 0.05

### Create mesh with gmsh

In [ ]:
class GenerateRectangleLines(object):
    def __init__(self):
        self.points = dict()
        self.lines = dict()

    def add_point(self, x, y):
        key = (x, y)
        try:
            return self.points[key]
        except KeyError:
            p = gmsh.model.geo.addPoint(x, y, 0.0, lcar)
            self.points[key] = p
            return p

    def add_line(self, p0, p1):
        try:
            return self.lines[(p0, p1)]
        except KeyError:
            l01 = gmsh.model.geo.addLine(p0, p1)
            self.lines[(p0, p1)] = l01
            self.lines[(p1, p0)] = -l01
            return l01

    def __call__(self, x_min, x_max, y_min, y_max):
        p0 = self.add_point(x_min, y_min)
        p1 = self.add_point(x_max, y_min)
        if isinstance(y_max, list):
            p2 = [self.add_point(x_max, y) for y in y_max]
            p3 = self.add_point(x_min, y_max[-1])
        else:
            p2 = self.add_point(x_max, y_max)
            p3 = self.add_point(x_min, y_max)
        l0 = self.add_line(p0, p1)
        if isinstance(y_max, list):
            p1_p2 = [p1] + p2
            l1 = [self.add_line(p1_p2[i], p1_p2[i + 1]) for i in range(len(p2))]
            l2 = self.add_line(p2[-1], p3)
        else:
            l1 = self.add_line(p1, p2)
            l2 = self.add_line(p2, p3)
        l3 = self.add_line(p3, p0)
        return (l0, l1, l2, l3)


gmsh.initialize()
gmsh.model.add("mesh")
generate_rectangle_lines = GenerateRectangleLines()
[l0, l1, l2, l3] = generate_rectangle_lines(0.0, 1.0, 0.0, [0.3, 0.7, 1.0])
[l4, l5, l6, _] = generate_rectangle_lines(1.0, 3.0, 0.0, 0.3)
[_, l7, l8, _] = generate_rectangle_lines(1.0, 3.0, 0.3, 0.7)
[_, l9, l10, _] = generate_rectangle_lines(1.0, 3.0, 0.7, 1.0)
line_loop_rectangle_left = gmsh.model.geo.addCurveLoop([l0, l1[0], l1[1], l1[2], l2, l3])
line_loop_rectangle_right_bottom = gmsh.model.geo.addCurveLoop([l4, l5, l6, -l1[0]])
line_loop_rectangle_right_middle = gmsh.model.geo.addCurveLoop([-l6, l7, l8, -l1[1]])
line_loop_rectangle_right_top = gmsh.model.geo.addCurveLoop([-l8, l9, l10, -l1[2]])
rectangle_left = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_left])
rectangle_right_bottom = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_right_bottom])
rectangle_right_middle = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_right_middle])
rectangle_right_top = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_right_top])
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0, l2, l3], 1)
gmsh.model.addPhysicalGroup(1, [l4, l10], 2)
gmsh.model.addPhysicalGroup(1, [l5, l7, l9], 3)
gmsh.model.addPhysicalGroup(2, [rectangle_left], 1)
gmsh.model.addPhysicalGroup(2, [rectangle_right_bottom, rectangle_right_top], 3)
gmsh.model.addPhysicalGroup(2, [rectangle_right_middle], 2)
gmsh.model.mesh.generate(2)

### Convert to a dolfinx mesh

In [ ]:
mesh, subdomains, boundaries = gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

### Save mesh, subdomains and boundaries

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "graetz_2.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)